In [ ]:
!wget -q -O distracted_driver.zip "https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=1"

In [ ]:
!unzip -q /content/distracted_driver.zip -d distracted_driver_data

In [ ]:
!unzip -q /content/distracted_driver_data/imgs.zip -d distracted_driver_data/imgs

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
#Required Paths for Train and Val data set
SOURCE_DIR = "distracted_driver_data/imgs/train"
SPLIT_DIR_BASE_PATH = "split_data"
TRAIN_DIR = os.path.join(SPLIT_DIR_BASE_PATH, "train")
VAL_DIR = os.path.join(SPLIT_DIR_BASE_PATH, "val")

In [ ]:

def count_images(dir) :
  # Dictionary to store counts
  class_counts = {}

  # Iterate through each class folder
  for class_name in sorted(os.listdir(dir)):
      class_path = os.path.join(dir, class_name)
      if os.path.isdir(class_path):
          count = len([
              file for file in os.listdir(class_path)
              if file.lower().endswith(('.jpg', '.jpeg', '.png'))
          ])
          class_counts[class_name] = count

  # Print results
  for class_name, count in class_counts.items():
      print(f"{class_name}: {count} images")

In [ ]:
count_images(SOURCE_DIR)

c0: 2489 images
c1: 2267 images
c2: 2317 images
c3: 2346 images
c4: 2326 images
c5: 2312 images
c6: 2325 images
c7: 2002 images
c8: 1911 images
c9: 2129 images


In [ ]:
# Create output folders
for base in [TRAIN_DIR, VAL_DIR]:
    for c in range(10):
        os.makedirs(os.path.join(base, f"c{c}"), exist_ok=True)

In [ ]:
image_paths = []
labels = []

In [ ]:
for label in os.listdir(SOURCE_DIR):
    class_dir = os.path.join(SOURCE_DIR, label)
    for img_file in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, img_file))
        labels.append(label)

In [ ]:
print(len(image_paths))

22424


In [ ]:
#Split the data into train and val
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

In [ ]:
for path, label in zip(train_paths, train_labels):
    shutil.copy(path, os.path.join(TRAIN_DIR, label, os.path.basename(path)))

In [ ]:
for path, label in zip(val_paths, val_labels):
    shutil.copy(path, os.path.join(VAL_DIR, label, os.path.basename(path)))

In [ ]:
print(f"Training samples: {len(train_paths)}, Validation samples: {len(val_paths)}")

Training samples: 17939, Validation samples: 4485


In [ ]:
TRAIN_DIR, VAL_DIR

('split_data/train', 'split_data/val')

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.regularizers import l2

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model

In [ ]:
def build_model(base_model):
  x = base_model.output
  global_pool = GlobalAveragePooling2D(name='global_avg_pooling')(x)
  bn1=BatchNormalization(name='bn1')(global_pool)
  dropout1 = Dropout(0.3, name='dropout1')(bn1)

  dense1 = Dense(128, activation='relu',name='dense1')(dropout1)
  bn2=BatchNormalization(name='bn2')(dense1)
  dropout2 = Dropout(0.4, name='dropout2')(bn2)

  output = Dense(10, activation='softmax', name='output')(dropout2)

  model = Model(inputs=base_model.input, outputs=output)

  return model

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
model=build_model(base_model)

In [ ]:
# model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_data_generator() :
  generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest');
  return generator

In [ ]:
def create_generator(d1,d2):

  #train_datagen = create_data_generator()
  #val_datagen = create_data_generator()

  train_generator = create_data_generator().flow_from_directory(TRAIN_DIR,
                                                    target_size=(d1,d2),
                                                    batch_size=BATCH_SIZE, #16
                                                    class_mode='categorical',
                                                    shuffle=False
                                                    )
  val_generator = create_data_generator().flow_from_directory(VAL_DIR,
                                                    target_size=(d1,d2),
                                                    batch_size=BATCH_SIZE, #16
                                                    class_mode='categorical')
  return train_generator,val_generator

In [ ]:
train_generator,val_generator = create_generator(*IMG_SIZE)

Found 17939 images belonging to 10 classes.
Found 4485 images belonging to 10 classes.


In [ ]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

steps_per_epoch, validation_steps

(560, 140)

In [ ]:
model.fit(train_generator,
                    # steps_per_epoch=steps_per_epoch,
                    epochs=10,
                    validation_data=val_generator,
                    # validation_steps=validation_steps
          )

Epoch 1/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 334s 566ms/step - accuracy: 0.1031 - loss: 3.1904 - val_accuracy: 0.1043 - val_loss: 2.4046
Epoch 2/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 311s 554ms/step - accuracy: 0.0973 - loss: 2.8789 - val_accuracy: 0.0990 - val_loss: 2.3903
Epoch 3/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 307s 547ms/step - accuracy: 0.1027 - loss: 2.7156 - val_accuracy: 0.1175 - val_loss: 2.3430
Epoch 4/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 304s 542ms/step - accuracy: 0.1017 - loss: 2.6297 - val_accuracy: 0.0999 - val_loss: 2.3329
Epoch 5/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 306s 545ms/step - accuracy: 0.0994 - loss: 2.5799 - val_accuracy: 0.1043 - val_loss: 2.3205
Epoch 6/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 305s 544ms/step - accuracy: 0.1036 - loss: 2.5200 - val_accuracy: 0.0941 - val_loss: 2.3189
Epoch 7/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 303s 540ms/step - accuracy: 0.1047 - loss: 2.4882 - val_accuracy: 0.1072 - val_loss: 2.3076
Epoch 8/10
561/561 ━━━━━━━━━━━━━━━━━━━━ 305s 544ms/step - accuracy: 0.0999 -

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Dictionary mapping class codes to descriptions
class_labels = {
    'c0': 'safe driving',
    'c1': 'texting - right',
    'c2': 'talking on the phone - right',
    'c3': 'texting - left',
    'c4': 'talking on the phone - left',
    'c5': 'operating the radio',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'hair and makeup',
    'c9': 'talking to passenger'
}

class_codes = {v: k for k, v in train_generator.class_indices.items()}

In [ ]:
def process_image_for_validation(my_image,d1,d2):
  img=image.load_img(my_image,target_size=(d1,d2))
  x=image.img_to_array(img)
  x=preprocess_input(x)
  x=np.expand_dims(x,axis=0)
  return x

In [ ]:
def predict_image(my_image,model,d1,d2):
  prediction = model.predict(process_image_for_validation(my_image,d1,d2))
  predicted_class = np.argmax(prediction)
  # class_codes = {v: k for k, v in train_generator.class_indices.items()}
  class_code = class_codes[predicted_class]
  class_label = class_labels[class_code]
  print(f"Image Path: {my_image}")
  print(f"predicted_class: {predicted_class}")
  print(f"Predicted Class: \"{class_code} : {class_label}\" with Probability: {prediction[0, predicted_class]:.2f}")

In [ ]:
images = []
images = ['/content/distracted_driver_data/imgs/test/img_1.jpg',
          '/content/distracted_driver_data/imgs/test/img_10.jpg',
          '/content/distracted_driver_data/imgs/test/img_100.jpg',
          '/content/distracted_driver_data/imgs/test/img_1000.jpg',
          '/content/distracted_driver_data/imgs/test/img_100000.jpg'
          ]

In [ ]:
for im in images:
  predict_image(im,model,*IMG_SIZE)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Image Path: /content/distracted_driver_data/imgs/test/img_1.jpg
predicted_class: 5
Predicted Class: "c5 : operating the radio" with Probability: 0.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Image Path: /content/distracted_driver_data/imgs/test/img_10.jpg
predicted_class: 0
Predicted Class: "c0 : safe driving" with Probability: 0.12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Image Path: /content/distracted_driver_data/imgs/test/img_100.jpg
predicted_class: 0
Predicted Class: "c0 : safe driving" with Probability: 0.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Image Path: /content/distracted_driver_data/imgs/test/img_1000.jpg
predicted_class: 0
Predicted Class: "c0 : safe driving" with Probability: 0.12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Image Path: /content/distracted_driver_data/imgs/test/img_100000.jpg
predicted_class: 4
Predicted Class: "c4 : talking on the phone - left" with Probability: 0.11
